In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import env
user = env.user
host = env.host
password = env.password

import os
os.path.exists('env.py')

def get_db_url(db, env_file=os.path.exists('env.py')):
    if env_file:
        user, password, host = (env.user, env.password, env.host)
        return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [2]:
# Given the following confusion matrix, evaluate (by hand) the model's performance.


# |               |   pred dog   |   pred cat   |
# |:------------  |  -----------:|  -----------:|
# | actual dog    |    TP     46 |   TN      7  |
# | actual cat    |    FP     13 |   FN      34 |

Accuracy = (46 + 7)/(46 + 13 + 7 + 34)
Precision = 46/(46 + 13)
Recall = 46/(46 + 34)
print('Accuracy ' ,Accuracy)
print('Precision ' ,round(Precision, 2))
print('Recall ' ,round(Recall, 2))

Accuracy  0.53
Precision  0.78
Recall  0.57


In [3]:
# pd.DataFrame(classification_report(df.actual, df.baseline_prediction, output_dict=True))
# print(classification_report(df.actual, df.baseline_prediction))

### In the context of this problem, what is a false positive?
- Pred Dog, Actual Cat

### In the context of this problem, what is a false negative?
- Pred Cat, Actual Cat

### How would you describe this model?
- This model has a overall poor accuracy and recall, the percision is about 78%.

### You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

### Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

### Use the predictions dataset and pandas to help answer the following questions:

- An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [6]:
c3 = pd.read_csv('c3.csv')
c3.head()

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect


In [8]:
pd.DataFrame(classification_report(c3.actual, c3.model1, output_dict=True))

,Defect,No Defect,accuracy,macro avg,weighted avg
precision,0.800000,0.957895,0.95,0.878947,0.945263
recall,0.500000,0.989130,0.95,0.744565,0.950000
f1-score,0.615385,0.973262,0.95,0.794323,0.944632
support,16.000000,184.000000,0.95,200.000000,200.000000


In [9]:
pd.DataFrame(classification_report(c3.actual, c3.model2, output_dict=True))

,Defect,No Defect,accuracy,macro avg,weighted avg
precision,0.100000,0.936364,0.56,0.518182,0.869455
recall,0.562500,0.559783,0.56,0.561141,0.560000
f1-score,0.169811,0.700680,0.56,0.435246,0.658211
support,16.000000,184.000000,0.56,200.000000,200.000000


In [10]:
pd.DataFrame(classification_report(c3.actual, c3.model3, output_dict=True))

,Defect,No Defect,accuracy,macro avg,weighted avg
precision,0.131313,0.970297,0.555,0.550805,0.903178
recall,0.812500,0.532609,0.555,0.672554,0.555000
f1-score,0.226087,0.687719,0.555,0.456903,0.650789
support,16.000000,184.000000,0.555,200.000000,200.000000


- Precision is the metric that is best for this example as we want to catch 'as many as possible'
- Model 1 is the best for this

Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?


- Recall is the metric that is best for this example as we want TP and FN are costly.
- Model 3 is best for this

### You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

### At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

### Several models have already been developed with the data, and you can find their results here.

### Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:



- In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

In [13]:
gives_you_paws = pd.read_csv('gives_you_paws.csv')
gives_you_paws

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog
...,...,...,...,...,...
4995,dog,dog,dog,dog,dog
4996,dog,dog,cat,cat,dog
4997,dog,cat,cat,dog,dog
4998,cat,cat,cat,cat,dog


In [12]:
gives_you_paws.mode()

,actual,model1,model2,model3,model4
0,dog,dog,cat,dog,dog


In [22]:
gives_you_paws.actual.value_counts() + gives_you_paws.model1.value_counts() + \
gives_you_paws.model2.value_counts() + gives_you_paws.model3.value_counts() + \
gives_you_paws.model4.value_counts()

cat    10260
dog    14740
dtype: int64

In [29]:
gives_you_paws['baseline'] = gives_you_paws.actual
gives_you_paws.baseline = gives_you_paws.baseline.map(dict(cat='dog', dog='dog'))
gives_you_paws

,actual,model1,model2,model3,model4,baseline
0,cat,cat,dog,cat,dog,dog
1,dog,dog,cat,cat,dog,dog
2,dog,cat,cat,cat,dog,dog
3,dog,dog,dog,cat,dog,dog
4,cat,cat,cat,dog,dog,dog
...,...,...,...,...,...,...
4995,dog,dog,dog,dog,dog,dog
4996,dog,dog,cat,cat,dog,dog
4997,dog,cat,cat,dog,dog,dog
4998,cat,cat,cat,cat,dog,dog


In [35]:
x = ['actual', 'model1', 'model2', 'model3', 'model4', 'baseline']
for col in x:
    print(pd.DataFrame(classification_report(gives_you_paws.actual, gives_you_paws[col], output_dict=True)))

              cat     dog  accuracy  macro avg  weighted avg
precision     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0       1.0        1.0           1.0
support    1746.0  3254.0       1.0     5000.0        5000.0
                   cat          dog  accuracy    macro avg  weighted avg
precision     0.689772     0.890024    0.8074     0.789898      0.820096
recall        0.815006     0.803319    0.8074     0.809162      0.807400
f1-score      0.747178     0.844452    0.8074     0.795815      0.810484
support    1746.000000  3254.000000    0.8074  5000.000000   5000.000000
                   cat          dog  accuracy    macro avg  weighted avg
precision     0.484122     0.893177    0.6304     0.688649      0.750335
recall        0.890607     0.490781    0.6304     0.690694      0.630400
f1-score      0.627269     0.633479    0.6304     0.630374      0.631310
support    1746.000000  3254.000000   

In [44]:
for col in x:
    print(col, (accuracy_score(gives_you_paws.actual, gives_you_paws[col])))

actual 1.0
model1 0.8074
model2 0.6304
model3 0.5096
model4 0.7426
baseline 0.6508


- In terms of accuracy the Model 1 and Model 4 is better then the baseline.

### Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recomend for Phase I? 

- Model 2 for the precision metric would best.

### Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recomend for Phase I?

- Model 2 for the precision metric would best.